### Import Libraries

In [ ]:
# Import the required packages and libraries
import datetime
import os
from dotenv import load_dotenv 
from pathlib import Path

In [ ]:
import json
import copy
import pandas as pd
from requests.exceptions import HTTPError

### Import Scripts

In [ ]:
from pyprediktormapclient.opc_ua import OPC_UA
from pyprediktormapclient.model_index import ModelIndex
from pyprediktormapclient.auth_client import AUTH_CLIENT
from pyprediktormapclient.analytics_helper import AnalyticsHelper
from pyprediktormapclient.shared import *

### Import Envrionment Variables

In [ ]:
# Consider obtaining the envrionment variables from .env file if you are running this locally from source.
dotenv_path = Path("../.env")
load_dotenv(dotenv_path=dotenv_path)

In [ ]:
username = os.environ["USERNAME"]
password = os.environ["PASSWORD"]
opcua_rest_url = os.environ["OPC_UA_REST_URL"]
opcua_server_url = os.environ["OPC_UA_SERVER_URL"]
model_index_url = os.environ["MODEL_INDEX_URL"]
ory_url = os.environ["ORY_URL"]

In [ ]:
# Getting ory bearer token
auth_client = AUTH_CLIENT(rest_url=ory_url, username=username, password=password)
auth_client.request_new_ory_token()

### Download data from modelindex api

In [ ]:
# Connecting to ModelIndex APIs 
model_data = ModelIndex(url=model_index_url, auth_client=auth_client, session=auth_client.session)

In [ ]:
# Listed sites on the model index api server
namespaces = model_data.get_namespace_array()
namespaces

In [ ]:
# Types of Objects
object_types_json = model_data.get_object_types()
object_types = AnalyticsHelper(object_types_json)
object_types.dataframe

In [ ]:
# Unique types of Objects
object_types_unique = object_types.dataframe[["Id", "Name"]].drop_duplicates()
object_types_unique

In [ ]:
# To get typeId by type name of an object
object_type_id = model_data.get_object_type_id_from_name("SiteType")
object_type_id

In [ ]:
# To get the objects of a type
sites_json = model_data.get_objects_of_type("SiteType")

# Send the returned JSON into a normalizer to get Id, Type, Name, Props and Vars as columns
sites = AnalyticsHelper(sites_json)
sites.list_of_names()

In [ ]:
# Analytics helper
sites.variables_as_dataframe()

In [ ]:
sites.list_of_ids()

In [ ]:
# Selecting the single site
site_id = sites.list_of_ids()[0]
site_id

In [ ]:
# Get all stringsets for one park
string_sets_for_first_park_as_json = model_data.get_object_descendants(
    "StringSetType", [site_id], "PV_Assets"
)
string_sets_for_first_park = AnalyticsHelper(string_sets_for_first_park_as_json)
string_sets_for_first_park.dataframe

In [ ]:
# Ancestors of an object type, get all trackers that are ancestor of the parks string sets

trackers_as_json = model_data.get_object_ancestors(
    "TrackerType", string_sets_for_first_park.list_of_ids(), "PV_Serves"
)
trackers = AnalyticsHelper(trackers_as_json)
trackers.variables_as_dataframe()

In [ ]:
sites_df = sites.variables_as_dataframe()
node_ids_list = sites_df['Id'].tolist()

In [ ]:
node_id_dicts = AnalyticsHelper.create_read_value_ids_list_for_event_types(sites)
node_id_dicts

In [ ]:
def create_read_value_ids_list_for_event_types(event_type_name_of_node_ids):
    """
    Create a list of NodeId dictionaries from a JSON response.

    The function extracts the 'Id' field from the JSON response, splits it into namespace, id type, and id,
    and creates a NodeId dictionary for each id. It then removes any duplicates from the list of NodeId dictionaries.
    """
    # Extracting Id column from the dataframe
    response_json = model_data.get_objects_of_type(event_type_name_of_node_ids)
    json_data = AnalyticsHelper(response_json)

    json_df = json_data.variables_as_dataframe()
    node_ids_list = json_df['Id'].tolist()

    node_id_dicts = []
    for id_str in node_ids_list:
        parts = id_str.split(":")
        node_id_dict = {
            "NodeId": {
                "Id": parts[2],
                "Namespace": int(parts[0]),
                "IdType": int(parts[1]),
            }
        }
        node_id_dicts.append(node_id_dict)

    node_id_dicts = [json.loads(t) for t in set(json.dumps(d) for d in node_id_dicts)]
    return node_id_dicts

In [ ]:
node_id_dicts = create_read_value_ids_list_for_event_types("SiteEventType")
node_id_dicts

### Download data from the opc ua api

In [ ]:
namespace_list = object_types.namespaces_as_list(namespaces)

# Initating the OPC UA API with a fixed namespace list
opc_data = OPC_UA(
    rest_url=opcua_rest_url, opcua_url=opcua_server_url, namespaces=namespace_list, auth_client=auth_client
)

In [ ]:
# Getting all event types
event_types = opc_data.get_event_types("SiteEventType")
event_types

In [ ]:
event_type_noded_id = "6:0:1070"

In [ ]:
def read_historical_events(opc_data,
    start_time,
    end_time,
    event_type_noded_id,
    node_id_dicts,
    fields_list= None,
    limit_start_index = None,
    limit_num_records = None,
    ) -> pd.DataFrame:

    #event_type_noded_id = opc_data.get_event_types(event_type_name)

    body = copy.deepcopy(opc_data.body)
    body["StartTime"] = start_time.strftime("%Y-%m-%dT%H:%M:%SZ")
    body["EndTime"] = end_time.strftime("%Y-%m-%dT%H:%M:%SZ")
    body["Fields"] = fields_list if fields_list else []
    body["WhereClause"] = {
            "EventTypeNodedId": {
                "Id": int(event_type_noded_id.split(":")[2]),
                "Namespace": int(event_type_noded_id.split(":")[0]),
                "IdType": int(event_type_noded_id.split(":")[1])
            }
        }
    body["ReadValueIds"] = node_id_dicts

    if limit_start_index is not None and limit_num_records is not None:
        body["Limit"] = {
            "StartIndex": limit_start_index,
            "NumRecords": limit_num_records
        }
    print(body)

    try:
        # Try making the request, if fails check if it is due to ory client
        content = request_from_api(
            rest_url=opcua_rest_url,
            method="POST",
            endpoint="events/read",
            data=json.dumps(body, default=opc_data.json_serial),
            headers=opc_data.headers,
            extended_timeout=True,
        )

    except HTTPError as e:
        if opc_data.auth_client is not None:
            opc_data.check_auth_client(json.loads(e.response.content))
        else:
            raise RuntimeError(f'Error message {e}')
    
    df_result = pd.json_normalize(content, record_path=["EventsResult"])
    df_hist_event = df_result.explode('HistoryEvents')
    df_hist_event_normalized = pd.json_normalize(df_hist_event['HistoryEvents'])
    df_hist_event_normalized = df_hist_event_normalized[fields_list]

    df_final = pd.concat([df_hist_event[df_hist_event.columns.difference(['HistoryEvents'])].reset_index(drop=True), df_hist_event_normalized.reset_index(drop=True)], axis=1)
    new_columns = fields_list + [col for col in df_final.columns if col not in fields_list]
    df_final = df_final[new_columns]
    df_final.rename(
            columns={
                "NodeId.Id": "Id",
                "NodeId.IdType": "IdType",
                "NodeId.Namespace": "Namespace",
                "StatusCode.Code": "StatusCode",
                "StatusCode.Symbol": "Quality",
            },
            errors="raise",
            inplace=True,
        )

    df_final.drop(columns=["IdType", "Namespace", "StatusCode", "Quality"], inplace=True)
    

    return df_final

In [ ]:
def read_historical_events1(opc_data,
    start_time,
    end_time,
    event_type_noded_id,
    fields_list= None,
    limit_start_index = None,
    limit_num_records = None,
    ) -> pd.DataFrame:

    event_type_noded_id 

    body = copy.deepcopy(opc_data.body)
    body["StartTime"] = start_time.strftime("%Y-%m-%dT%H:%M:%SZ")
    body["EndTime"] = end_time.strftime("%Y-%m-%dT%H:%M:%SZ")
    body["Fields"] = fields_list if fields_list else []
    body["WhereClause"] = {
            "EventTypeNodedId": {
                "Id": int(event_type_noded_id.split(":")[2]),
                "Namespace": int(event_type_noded_id.split(":")[0]),
                "IdType": int(event_type_noded_id.split(":")[1])
            }
        }
    body["ReadValueIds"] = [
        {
        "NodeId": {
            'Id': 'Enterprise.EG-AS',
            'Namespace': 3,
            'IdType': 1
        }
        }
    ]

    if limit_start_index is not None and limit_num_records is not None:
        body["Limit"] = {
            "StartIndex": limit_start_index,
            "NumRecords": limit_num_records
        }
    print(body)

    try:
        # Try making the request, if fails check if it is due to ory client
        content = request_from_api(
            rest_url=opcua_rest_url,
            method="POST",
            endpoint="events/read",
            data=json.dumps(body, default=opc_data.json_serial),
            headers=opc_data.headers,
            extended_timeout=True,
        )

    except HTTPError as e:
        if opc_data.auth_client is not None:
            opc_data.check_auth_client(json.loads(e.response.content))
        else:
            raise RuntimeError(f'Error message {e}')
    
    df_result = pd.json_normalize(content, record_path=["EventsResult"])
    df_hist_event = df_result.explode('HistoryEvents')
    df_hist_event_normalized = pd.json_normalize(df_hist_event['HistoryEvents'])
    df_hist_event_normalized = df_hist_event_normalized[fields_list]

    df_final = pd.concat([df_hist_event[df_hist_event.columns.difference(['HistoryEvents'])].reset_index(drop=True), df_hist_event_normalized.reset_index(drop=True)], axis=1)
    new_columns = fields_list + [col for col in df_final.columns if col not in fields_list]
    df_final = df_final[new_columns]
    df_final.rename(
            columns={
                "NodeId.Id": "Id",
                "NodeId.IdType": "IdType",
                "NodeId.Namespace": "Namespace",
                "StatusCode.Code": "StatusCode",
                "StatusCode.Symbol": "Quality",
            },
            errors="raise",
            inplace=True,
        )

    df_final.drop(columns=["IdType", "Namespace", "StatusCode", "Quality"], inplace=True)
    

    return df_final

In [ ]:
event_type_noded_id = "6:0:1070"
start_time=(datetime.datetime.now() - datetime.timedelta(1))
end_time=(datetime.datetime.now() - datetime.timedelta(30))

In [ ]:
# Reading historical events data 
hist_events = read_historical_events1(opc_data,
    start_time,
    end_time,
    event_type_noded_id,
    fields_list = ["Time", "Message", "Severity", "SourceName"],
)
hist_events

In [ ]:
# Reading one month historical events of trackers 
hist_events = opc_data.read_historical_events(
    start_time=(datetime.datetime.now() - datetime.timedelta(30)),
    end_time=(datetime.datetime.now() - datetime.timedelta(1)),
    event_type_name="InverterEventType",
    event_type_node_ids = node_ids_list,
    fields_list = ["Time", "Message", "Severity", "SourceName"]
)
hist_events

In [ ]:
# Live value data of trackers
live_value = opc_data.get_values(
    inverters.variables_as_list(["DCPower"])
)
live_value

In [ ]:
# Live value data of trackers
live_value = opc_data.get_values(
    trackers.variables_as_list(["AngleMeasured"])
)
live_value

In [ ]:
# Historic value data of trackers, 1 days worth of data 30 days ago
one_day_historic_tracker_data = opc_data.get_historical_aggregated_values(
    start_time=(datetime.datetime.now() - datetime.timedelta(30)),
    end_time=(datetime.datetime.now() - datetime.timedelta(29)),
    pro_interval=3600000,
    agg_name="Average",
    variable_list=trackers.variables_as_list(["AngleMeasured"]),
)
one_day_historic_tracker_data

In [ ]:
one_day_historic_tracker_data["Id"].unique()

In [ ]:
data = trackers.variables_as_list()
data

In [ ]:
data = trackers.variables_as_list()
# Extract the 'Id' values that end with '.Signals.DCPower'
angle_measured_ids = [item['Id'] for item in data if item['Id'].endswith('.Signals.AngleMeasured')]

unique_dc_power_ids = set(dc_power_ids)
unique_dc_power_ids

In [ ]:
len(unique_dc_power_ids)